In [0]:
import pandas as pd
import numpy as np
import os
import sys
import sklearn
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm

User uploaded file "ACC_AUX.csv" with length 3201698 bytes

User uploaded file "ACCIDENT.csv" with length 5398194 bytes

User uploaded file "CEVENT.csv" with length 2625888 bytes

User uploaded file "DAMAGE.csv" with length 3063538 bytes

User uploaded file "DISTRACT.csv" with length 792596 bytes

User uploaded file "DRIMPAIR.csv" with length 780534 bytes

User uploaded file "DRUGS.csv" with length 1883861 bytes

User uploaded file "FACTOR.csv" with length 765741 bytes

User uploaded file "MANEUVER.csv" with length 795168 bytes

User uploaded file "MIACC.csv" with length 1041740 bytes

User uploaded file "MIDRVACC.csv" with length 1021371 bytes

User uploaded file "MIPER.csv" with length 1998730 bytes

User uploaded file "NMCRASH.csv" with length 180454 bytes

User uploaded file "NMIMPAIR.csv" with length 134464 bytes

User uploaded file "NMPRIOR.csv" with length 135301 bytes

User uploaded file "PARKWORK.csv" with length 267236 bytes

User uploaded file "PBTYPE.csv" with length 462439 bytes

User uploaded file "PER_AUX.csv" with length 4792170 bytes

User uploaded file "PERSON.csv" with length 13256237 bytes

User uploaded file "SAFETYEQ.csv" with length 207131 bytes

User uploaded file "VEH_AUX.csv" with length 2324544 bytes

User uploaded file "VEHICLE.csv" with length 16363292 bytes

User uploaded file "VEVENT.csv" with length 3567752 bytes

User uploaded file "VINDECODE.csv" with length 24128934 bytes

User uploaded file "VIOLATN.csv" with length 811167 bytes

User uploaded file "VISION.csv" with length 761725 bytes

User uploaded file "VSOE.csv" with length 2317084 bytes

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
df = pd.read_csv("PERSON.csv") # For on Colab
# df = pd.read_csv("FARS2018NationalCSV\\PERSON.csv") # For local jupyter

In [0]:
df

In [0]:
df2 = df[['HOUR','RUR_URB','FUNC_SYS','HARM_EV','MAN_COLL','BODY_TYP','MOD_YEAR','ROLLOVER','IMPACT1','FIRE_EXP','AGE','SEX','SEAT_POS','REST_USE','AIR_BAG','EJECTION','EXTRICAT','INJ_SEV']]
df2.columns = [x.lower() for x in df2.columns]
df2 = df2.rename(columns = {'func_sys':'road_type',
                            'harm_ev':'harm_event',
                            'man_coll':'car_orientation',
                            'body_typ':'body_type',
                            'mod_year':'model_year',
                            'impact1':'impact_area',
                            'fire_exp':'fire',
                            'rest_use':'protection',
                            'air_bag':'airbag',
                            'extricat':'extricated'})
df2 = df2[df2['hour'] < 24]
df2 = df2[df2['rur_urb'] < 3]
df2 = df2[df2['road_type'] < 8]
df2 = df2[df2['harm_event'] < 99]
df2 = df2[df2['car_orientation'] < 11]
df2 = df2[df2['body_type'] < 91]
df2 = df2[df2['model_year'] < 9998]
df2 = df2[df2['impact_area'] < 98]
df2 = df2[df2['age'] < 998]
df2 = df2[df2['sex'] < 8]
df2 = df2[df2['seat_pos'] < 98]
df2 = df2[df2['protection'] < 29]
df2 = df2[df2['airbag'] < 98]
df2 = df2[(df2['ejection'] != 9) & (df2['ejection'] != 7)]
df2 = df2[df2['extricated'] < 9]
df2 = df2[df2['inj_sev'] < 5]
df2 = df2.astype('int64')
df2 = df2.reset_index(drop = True)

In [0]:
df2.columns

In [0]:
df2['extricated'].value_counts().sort_index()

In [2]:
dummy_df = pd.get_dummies(df2, columns=['rur_urb','road_type','harm_event','car_orientation','body_type','rollover','impact_area','seat_pos','protection','airbag', 'ejection'], drop_first=True)

NameError: ignored

In [3]:
dummy_df

NameError: ignored

In [4]:
Xcols = dummy_df.columns.tolist()

NameError: ignored

In [5]:
Xcols.remove('inj_sev')

NameError: ignored

In [0]:
X = dummy_df[Xcols].values
y = dummy_df['inj_sev'].values
# Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .2, stratify=y, random_state = 50)

In [0]:
pca = PCA(n_components=10)
pca.fit(X)
feature_pca = pca.transform(X)

In [0]:
feature_pca

array([[ 1.51369316e+01, -1.06802508e+01,  6.74738737e+00, ...,
        -7.67349616e-03, -3.20879363e-01,  5.81744880e-03],
       [-3.85904609e+00,  7.06083436e+00, -8.42824960e+00, ...,
        -1.32480846e-01, -2.68753053e-01, -1.56576669e-01],
       [ 1.81332951e+01,  3.63904778e+00,  4.46646297e-01, ...,
        -6.90942996e-01, -6.56957507e-01, -5.20250607e-01],
       ...,
       [ 2.11194145e+01,  1.38929062e+01, -4.03205784e+00, ...,
         1.64214850e-01, -1.95169028e-02, -3.99838960e-02],
       [-5.87539634e+00,  5.60672381e+00,  5.08206189e-01, ...,
        -9.20807454e-02, -2.89725519e-01,  2.77036734e-01],
       [ 1.18136442e-01,  1.58568778e+01, -3.97710149e+00, ...,
        -4.32310631e-01, -4.62142070e-01,  5.80731162e-02]])

In [0]:
model = LinearSVC(C = 20, verbose=99)
pipe_steps = [('pca',PCA(n_components=10)), ('linearsvc',LinearSVC())]

check_params = {
    'pca__n_components': [8,10,12],
    'linearsvc__C': [.1, .5, 1, 10, 20, 30, 40, 50, 75, 100, 500, 1000]
}
# scores = sklearn.model_selection.cross_val_score(model, feature_pca, y, cv=5, verbose = 99, scoring='accuracy')
pipeline = Pipeline(pipe_steps)

# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [0]:
create_grid = GridSearchCV(pipeline, param_grid=check_params, cv=5, scoring = 'accuracy')
create_grid.fit(Xtrain, ytrain)
print(create_grid.score(Xtest, ytest))
print(create_grid.best_params_)

C:\Users\ericm\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ericm\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ericm\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ericm\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ericm\AppData\Local\Programs\Python\Python37\lib\site-packa

In [1]:
import statsmodels.api as sm

model = sm.GLM(y, X, axis = 1, family=sm.families.Binomial())

results = model.fit()

print(results.summary())

NameError: ignored